In [1]:
import os, sys

from math import ceil
import pyvista as pv
import numpy as np
import meshplot as mp
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
from skimage import measure
from scipy.ndimage import zoom
from scipy.interpolate import interpn
from IPython.display import display
from einops import rearrange
import igl
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import torch
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import open3d as o3d
from IPython.display import display
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Meshplot left an annoying print statement in their code. Using this context manager to supress it...
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [12]:
# Dot product on the first dimension of n-dimensional arrays x and y
def dot(x, y):
    return np.einsum('i..., i... -> ...', x, y)

# Signed distance functions from Inigo Quilez https://iquilezles.org/articles/distfunctions/
# You could implement the smooth minimum operation as well to compose shapes together for more complex situations
def sdf_sphere(x, radius):
    return np.linalg.norm(x, axis=0) - radius

def sdf_capsule(x, a, b, r):
    xa = coords - a
    ba = coords - a
    h = np.clip(dot(xa, ba) / dot(ba, ba), 0., 1.)
    return np.linalg.norm(xa - ba * h) - r

def sdf_torus(x, radius, thickness):
    
    q = np.stack([np.linalg.norm(x[[0, 1]], axis=0) - radius, x[2]])
    return np.linalg.norm(q, axis=0) - thickness

# Crop an n-dimensional image with a centered cropping region
def center_crop(img, shape):
    start = [a // 2 - da // 2 for a, da in zip(img.shape, shape)]
    end = [a + b for a, b in zip(start, shape)]
    slices = tuple([slice(a, b) for a, b in zip(start, end)])
    return img[slices]

# Add noise to coordinates
def gradient_noise(x, scale, strength, seed=None):
    shape = [ceil(s / scale) for s in x.shape[1:]]
    if seed:
        np.random.seed(seed)
    scalar_noise = np.random.randn(*shape)
    scalar_noise = zoom(scalar_noise, zoom=scale)
    scalar_noise = center_crop(scalar_noise, shape=x.shape[1:])
    vector_noise = np.stack(np.gradient(scalar_noise))
    return vector_noise * strength


In [10]:
plot=None
@mp.interact(
    radius=(0, 0.3, 0.01), 
    thickness=(0.01, 0.1, 0.01), 
    noise_scale=(5, 25), 
    noise_strength=(0.0, 0.4, 0.05),
    seed=(1, 100)
)
def show(radius, thickness, noise_scale, noise_strength, seed):
    global plot
    global sdf
    coords = np.linspace(-1, 1, 100)
    x = np.stack(np.meshgrid(coords, coords, coords))
    x = x + gradient_noise(x, noise_scale, noise_strength, seed)
    sdf = sdf_torus(x, radius, thickness)
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)
    
    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

interactive(children=(FloatSlider(value=0.15, description='radius', max=0.3, step=0.01), FloatSlider(value=0.0…

In [11]:
plot=None
@mp.interact(
    radius=(0, 0.5, 0.01), 
    thickness=(0.01, 0.20, 0.01), 
    noise_scale=(0.0, 20, 1),
    noise_strength=(0.0, 10, 1),
    seed=(1, 100),
    bump_angle=(-1., 1., 0.01),
    bump_width=(0.01, 0.02, 0.001),
    bump_height=(0.01, 50.),
    scale=(0.85, 1.15, 0.01),
)
def show(radius, thickness, noise_scale, noise_strength, seed, bump_angle, bump_width, bump_height, scale):
    global plot
    coords = np.linspace(-1, 1, 100)
    x = np.stack(np.meshgrid(coords, coords, coords))
    sdf = sdf_torus(x, radius, thickness)
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)
    
    x_warp = gradient_noise(x, noise_scale, noise_strength, seed)
    print(x_warp.shape)
    
    angle = np.pi * bump_angle
    gaussian_center = np.array([np.cos(angle), np.sin(angle), 0.]) * radius
    print(gaussian_center)
    x_dist = np.linalg.norm((x - gaussian_center[:, None, None, None]), axis=0)
    print(x_dist.shape)
    x_bump = bump_height * np.e ** -(1. / bump_width * x_dist ** 2)
    print(x_bump.shape)
    x_warp += -np.stack(np.gradient(x_bump))
    
    x_warp = rearrange(x_warp, 'v h w d -> h w d v')
    vertex_noise = interpn([np.arange(100) for _ in range(3)], x_warp, verts)
    verts += vertex_noise
    
    original_center = np.mean(verts, axis=0)
    
    verts = verts*scale
    
    new_center = np.mean(verts, axis=0)
    displacement_vector = original_center - new_center
    verts += displacement_vector
    
    print(verts.shape)
    print(faces.shape)
    max_values = np.amax(verts, axis=0)
    min_values = np.amin(verts, axis=0)

    print("Maximum values:", max_values)
    print("Minimum values:", min_values)
    if plot is None:
        plot = mp.plot(verts, faces, return_plot=True)
    else:
        with HiddenPrints():
            plot.update_object(vertices=verts, faces=faces)
        display(plot._renderer)

interactive(children=(FloatSlider(value=0.25, description='radius', max=0.5, step=0.01), FloatSlider(value=0.0…

## Save torus data into file

In [ ]:
radius=0.25 
thickness=0.10
noise_scale=16
noise_strength=10
seed=50
bump_width=.01
bump_height=25

feature_range_bump_height = np.linspace(20, 45, 300, endpoint=False)
feature_range_scale = np.linspace(0.85, 1.15, 300, endpoint=False)
feature_range_angle = np.linspace(-1, 1, 5000, endpoint=False)

scaler = MinMaxScaler()
scaler.fit(feature_range_angle.reshape(-1, 1))

scaler_s = MinMaxScaler()
scaler_s.fit(feature_range_scale.reshape(-1, 1))

scaler_b = MinMaxScaler()
scaler_b.fit(feature_range_bump_height.reshape(-1, 1))

labels = {}

for idx, bump_angle in tqdm(enumerate(np.linspace(-1, 1, 5000))):
    coords = np.linspace(-1, 1, 100)
    x = np.stack(np.meshgrid(coords, coords, coords))
    
    filepath = f"torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/torus_bump_{idx:04d}.ply"
    filename = filepath.split("/")[-1].split(".")[0]
    
    scale = random.choice(feature_range_scale)
    bump_height = random.choice(feature_range_bump_height)
    print(scale)
    print(bump_height)
    sdf = sdf_torus(x, radius, thickness)
    
    
    verts, faces, normals, values = measure.marching_cubes(sdf, level=0)  
    
    s = scaler_s.transform(np.array([scale]).reshape(-1,1)).item()
    b = scaler_b.transform(np.array([bump_height]).reshape(-1,1)).item()
    
    bump_width = 0.001 if idx % 2 == 0 else 0.01
    bump_angle = 1
    
    labels[filename] = np.array([idx % 2, bump_angle, s, b])
    
    print(labels[filename])
    print(len(verts))
    print(len(faces))

    x_warp = gradient_noise(x, noise_scale, noise_strength) #### no seed, random noise
    angle = np.pi * bump_angle
    gaussian_center = np.array([np.cos(angle), np.sin(angle), 0]) * radius
    x_dist = np.linalg.norm((x - gaussian_center[:, None, None, None]), axis=0)
    x_bump = bump_height * np.e ** -(1. / bump_width * x_dist ** 2)

    x_warp += -np.stack(np.gradient(x_bump))

    x_warp = rearrange(x_warp, 'v h w d -> h w d v')
    vertex_noise = interpn([np.arange(100) for _ in range(3)], x_warp, verts)
   
    verts += vertex_noise
    
    original_center = np.mean(verts, axis=0)
    
    verts = verts*scale
    
    new_center = np.mean(verts, axis=0)
    displacement_vector = original_center - new_center
    verts += displacement_vector

    igl.write_triangle_mesh(filepath, verts, faces)
    
torch.save(labels, "torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/labels.pt")


# Create an empty DataFrame
df = pd.DataFrame(columns=['shape', 'is_bump', 'angle', 'scale', 'bump_height'])

# Iterate over the items in the labels dictionary
for shape, values in labels.items():
    # Access the individual values from the NumPy array
    value1 = values[0]
    value2 = values[1]
    value3 = values[2]
    value4 = values[3]

    # Append a new row with the values to the DataFrame
    df = df.append({'shape': shape, 'is_bump': value1, 'angle': value2, 'scale': value3, 'bump_height': value4}, ignore_index=True)

# Write the DataFrame to an Excel file
df.to_csv("torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/torus_bump_labels.csv")

In [292]:
scaler_t = MinMaxScaler()
scaler_t.fit(feature_range_thickness.reshape(-1, 1))
t = scaler_t.transform(np.array([0.1]).reshape(-1,1)).item()
t

0.5084745762711864

## Save template into file

In [5]:
radius=0.25
thickness=0.10
noise_scale=16
noise_strength=10
seed=50
bump_width=0.001
bump_height=0.001
bump_angle = 1

coords = np.linspace(-1, 1, 100)
x = np.stack(np.meshgrid(coords, coords, coords))
sdf = sdf_torus(x, radius, thickness)
verts, faces, normals, values = measure.marching_cubes(sdf, level=0)    

x_warp = gradient_noise(x, noise_scale, noise_strength) # no seed, random noise

angle = np.pi * bump_angle
gaussian_center = gaussian_center = np.array([np.cos(angle), np.sin(angle), 0]) * radius
x_dist = np.linalg.norm((x - gaussian_center[:, None, None, None]), axis=0)
x_bump = bump_height * np.e ** -(1. / bump_width * x_dist ** 2)
x_warp += -np.stack(np.gradient(x_bump))

x_warp = rearrange(x_warp, 'v h w d -> h w d v')
vertex_noise = interpn([np.arange(100) for _ in range(3)], x_warp, verts)
verts += vertex_noise

#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#verts = scaler.fit_transform(verts)
print(verts.shape)
print(faces.shape)

igl.write_triangle_mesh(f"torus_bump_5000_two_scale_binary_bump_variable_noise_fixed_angle/template/template.ply", verts, faces)

(3496, 3)
(6992, 3)


True

## Save (min-max-scaled) labels into file

In [ ]:
scaler = MinMaxScaler()
labels = scaler.fit_transform(np.linspace(0, 1, 500).reshape(-1,1))
torch.save(labels.reshape(-1), "torus_bump_500_two/labels.pt")

In [459]:
labels = {}
feature_range = np.linspace(-1, 1, 500)
i=0
for x in tqdm(feature_range):
    filename = f'torus_bump_{i}'
    labels[filename] = x
    i = i+1

torch.save(labels, "C:/Users/Jakar/Downloads/Hippocampus_Study/torus_bump_500/torus_bump_500/labels.pt")
pd.DataFrame(list(labels.items()), columns=['shape', 'label']).to_csv("C:/Users/Jakar/Downloads/Hippocampus_Study/torus_bump_500/torus_bump_500/torus_bump_labels.csv")

100%|██████████| 500/500 [00:00<?, ?it/s]
